In [75]:
import xml.etree.ElementTree

import pandas as pd
import requests
from xml.etree import ElementTree as ET
import datetime
import json
from typing import List, Optional
from dateutil import parser
import urllib.parse as urlparse


In [96]:
dict(hello=1, we=2)

{'hello': 1, 'we': 2}

In [28]:
data = requests.get("https://assignments.reaktor.com/birdnest/drones")

In [34]:
data = requests.get("https://assignments.reaktor.com/birdnest/drones")
root = ET.fromstring(data.content)
for child in root:
    print(child.tag, child.attrib, child)

deviceInformation {'deviceId': 'GUARDB1RD'} <Element 'deviceInformation' at 0x12abb6900>
capture {'snapshotTimestamp': '2023-01-14T19:08:21.549Z'} <Element 'capture' at 0x12abb62c0>


In [77]:
"""
Data Fetcher and Data Parser Classes
"""
CENTER_X = 250000 # 250*1000
CENTER_Y = 250000
RADIUS = 100000


class BadResponseFromUrlException(Exception):
    """
    Exception raised from errors in any Data Query for this application
    """

    def __init__(self, status_code, url):
        self.status_code = status_code
        self.url = url
        super().__init__("Following URL returned bad status code\nStatus Code: {}\nURL used: {}".format(status_code,
                                                                                                        url))


class ViolatedPilotInformation:
    def __init__(self,
                 pilot_id: str,
                 first_name: str,
                 last_name: str,
                 phone_number: str,
                 created_dt: datetime.datetime,
                 email: str):
        self.pilot_id = pilot_id
        self.first_name = first_name
        self.last_name = last_name
        self.phone_number = phone_number
        self.created_dt = created_dt
        self.email = email

    @classmethod
    def from_dict(cls,
                  data_dict: dict) -> 'ViolatedPilotInformation':
        """
        Example of incoming
        {'pilotId': 'P-7qxuJaBYUa',
         'firstName': 'Roosevelt',
         'lastName': 'Fadel',
         'phoneNumber': '+210499153597',
         'createdDt': '2023-01-11T07:25:09.230Z',
         'email': 'roosevelt.fadel@example.com'}
        :return:
        """
        return ViolatedPilotInformation(
            pilot_id=data_dict['pilotId'],
            first_name=data_dict['firstName'],
            last_name=data_dict['lastName'],
            phone_number=data_dict['phoneNumber'],
            created_dt=parser.parse(data_dict['createdDt']),
            email=data_dict['email']
        )


class DroneInformation:
    def __init__(self,
                 serial_number: str,
                 model: str,
                 manufacturer: str,
                 mac: str,
                 ipv4: str,
                 ipv6: str,
                 firmware: str,
                 position_x: float,
                 position_y: float,
                 altitude: float,
                 snapshot_timestamp: datetime.datetime):
        self.serial_number = serial_number
        self.model = model
        self.manufacturer = manufacturer
        self.mac = mac
        self.ipv4 = ipv4
        self.ipv6 = ipv6
        self.firmware = firmware
        self.position_x = position_x
        self.position_y = position_y
        self.altitude = altitude
        self.snapshot_timestamp = snapshot_timestamp
        self.violated_ndz = self.is_violating_ndz(self.position_x, self.position_y)
        self.pilot_information: Optional[ViolatedPilotInformation] = None

    def set_pilot_information(self, pilot: ViolatedPilotInformation):
        self.pilot_information = pilot

    @staticmethod
    def is_violating_ndz(x, y, center_x=CENTER_X, center_y=CENTER_Y, radius=RADIUS):
        """
        static method top verify if the drone is violating the ndz.
        the logic is simple, from origin position 250000, 250000
        if location satisfy (x - center_x)² + (y - center_y)² <= radius²
        we will return True, otherwise False
        500000
        :param x:
        :param y:
        :param center_x:
        :param center_y:
        :param radius:
        :return:
        """
        return (x - center_x) ** 2 + (y - center_y) ** 2 <= radius ** 2

    @classmethod
    def from_xml_node(cls, node: ET.Element, snapshot_timestamp: datetime.datetime) -> 'DroneInformation':
        if node.tag != "drone":
            raise Exception('Invalid Node: Given Node is not drone Node.')
        data = {}
        for child in node:
            data[child.tag] = child.text
        return DroneInformation(
            serial_number=data['serialNumber'],
            model=data['model'],
            manufacturer=data['manufacturer'],
            mac=data['mac'],
            ipv4=data['ipv4'],
            ipv6=data['ipv6'],
            firmware=data['firmware'],
            position_x=float(data['positionX']),
            position_y=float(data['positionY']),
            altitude=float(data['altitude']),
            snapshot_timestamp=snapshot_timestamp
        )


class DroneCollection:
    def __init__(self,
                 drones: List[DroneInformation]):
        self.drones = drones

    @classmethod
    def from_xml_node(cls, node: ET.Element) -> 'DroneCollection':
        if node.tag != "capture":
            raise Exception('Invalid Node: Given Node is not capture Node.')
        data = {'snapshotTimestamp': node.attrib['snapshotTimestamp'],
                'drones': []}
        data['snapshotTimestamp'] = parser.parse(data['snapshotTimestamp'])
        for child in node:
            data['drones'].append(DroneInformation.from_xml_node(child, data['snapshotTimestamp']))
        return DroneCollection(
            drones=data['drones']
        )


class DeviceInformation:
    def __init__(self,
                 device_id: str,
                 listen_range: int,
                 device_started: datetime.datetime,
                 update_interval_ms: int):
        self.device_started = device_started
        self.listen_range = listen_range
        self.device_id = device_id
        self.update_interval_ms = update_interval_ms

    @classmethod
    def from_xml_node(cls, node: ET.Element) -> 'DeviceInformation':
        """
        class method to create Device Information Object from xml element with deviceInformation tag
        :param node: xml etree element with deviceInformation
        :return:
        """
        if node.tag != "deviceInformation":
            raise Exception('Invalid Node: Given Node is not deviceInformation Node.')
        data = {'deviceId': node.attrib['deviceId']}
        for child in node:
            data[child.tag] = child.text
        return DeviceInformation(
            device_id=data['deviceId'],
            listen_range=int(data['listenRange']),
            device_started=parser.parse(data['deviceStarted']),
            update_interval_ms=int(data['updateIntervalMs'])
        )

In [78]:
'''
DataFetcher Class
'''
class DataFetcher:
    def __init__(self,
                 drone_url='https://assignments.reaktor.com/birdnest/drones',
                 pilot_url='https://assignments.reaktor.com/birdnest/pilots/'):
        response = requests.get(drone_url)
        if 200 > response.status_code > 299:
            raise BadResponseFromUrlException(response.status_code, drone_url)
        root = ET.fromstring(response.content)
        parsed_dict = {}
        for child in root:
            if child.tag == 'deviceInformation':
                parsed_dict['deviceInformation'] = DeviceInformation.from_xml_node(child)
            elif child.tag == 'capture':
                parsed_dict['capture'] = DroneCollection.from_xml_node(child)
        self.drone_url = drone_url
        self.device_information = parsed_dict['deviceInformation']
        self.drone_collection = parsed_dict['capture']
        self.violated_pilot_data_fetcher = ViolatedPilotDataFetcher(pilot_url)
        for drone in self.drone_collection.drones:
            if drone.violated_ndz:
                pilot = self.violated_pilot_data_fetcher.query_violated_pilot_data(drone)
                drone.set_pilot_information(pilot)


class ViolatedPilotDataFetcher:
    def __init__(self, pilot_url='https://assignments.reaktor.com/birdnest/pilots/'):
        self.pilot_url = pilot_url

    def query_violated_pilot_data(self,
                                  drone: DroneInformation) -> ViolatedPilotInformation:
        query_url = self.pilot_url + drone.serial_number
        response = requests.get(urlparse.urlparse(query_url).geturl())  # parse url for cleaning
        if 200 > response.status_code > 299:
            raise BadResponseFromUrlException(response.status_code, self.pilot_url)
        result = json.loads(response.content)
        return ViolatedPilotInformation.from_dict(result)

In [ ]:
'''
SQL alchemy send data out and update

'''

In [71]:
"""Viewer Classes"""

'Viewer Classes'

In [92]:
datafetcher = DataFetcher()

In [95]:
print(len(datafetcher.drone_collection.drones))
for drone in datafetcher.drone_collection.drones:
    print(drone.serial_number, drone.violated_ndz, drone.pilot_information)
    if drone.violated_ndz:
        print(drone.pilot_information.pilot_id, drone.pilot_information.first_name)
        print(drone.pilot_information.)

3
SN-_hIr1kgkni True <__main__.ViolatedPilotInformation object at 0x13a0ebca0>
P-_JSj-lgzfg Callie
SN-VS7aaVjA26 False None
SN-qNDGgeFVkh False None


In [99]:
import math

math.sqrt((321500.69714086375-CENTER_X)**2 + (248687.5198673108-CENTER_Y)**2) < 100000

True

In [54]:
datafetcher.drone_collection.drones[0].serial_number

'SN-xnZhawf0eK'

In [59]:
data = requests.get("https://assignments.reaktor.com/birdnest/pilots/SN-xnZhawf0eK")
json.loads(data.content)

{'pilotId': 'P-7qxuJaBYUa',
 'firstName': 'Roosevelt',
 'lastName': 'Fadel',
 'phoneNumber': '+210499153597',
 'createdDt': '2023-01-11T07:25:09.230Z',
 'email': 'roosevelt.fadel@example.com'}

In [57]:
data.content

b'{"pilotId":"P-7qxuJaBYUa","firstName":"Roosevelt","lastName":"Fadel","phoneNumber":"+210499153597","createdDt":"2023-01-11T07:25:09.230Z","email":"roosevelt.fadel@example.com"}'

In [27]:
tree.get('report')

In [20]:
pd.read_xml(data.content)

,deviceId,listenRange,deviceStarted,uptimeSeconds,updateIntervalMs,snapshotTimestamp,drone
0,GUARDB1RD,500000.0,2023-01-14T16:54:41.057Z,7888.0,2000.0,None,NaN
1,None,NaN,None,NaN,NaN,2023-01-14T19:06:09.445Z,NaN


In [29]:
print(data.content)

b'<?xml version="1.0" encoding="UTF-8"?>\n<report>\n  <deviceInformation deviceId="GUARDB1RD">\n    <listenRange>500000</listenRange>\n    <deviceStarted>2023-01-14T16:54:41.057Z</deviceStarted>\n    <uptimeSeconds>8021</uptimeSeconds>\n    <updateIntervalMs>2000</updateIntervalMs>\n  </deviceInformation>\n  <capture snapshotTimestamp="2023-01-14T19:08:21.549Z">\n    <drone>\n      <serialNumber>SN-OCHqp2E4Aj</serialNumber>\n      <model>Mosquito</model>\n      <manufacturer>MegaBuzzer Corp</manufacturer>\n      <mac>82:e6:4e:65:85:a7</mac>\n      <ipv4>146.249.207.86</ipv4>\n      <ipv6>b360:671a:e339:1b03:a16e:d162:a27a:5370</ipv6>\n      <firmware>9.0.9</firmware>\n      <positionY>241087.73231406466</positionY>\n      <positionX>10712.865762471618</positionX>\n      <altitude>4246.835465187369</altitude>\n    </drone>\n    <drone>\n      <serialNumber>SN-NzGYw1YoGy</serialNumber>\n      <model>Altitude X</model>\n      <manufacturer>DroneGoat Inc</manufacturer>\n      <mac>da:ea:7d